# Chapter11. 심층 신경망 훈련

## 심층 신경망 훈련 중 마주할 수 있는 문제
1. 그레디언트 소실/그레디언트 폭주
    - 심층 신경망의 아래쪽으로 갈수록 그레디언트가 점점 작아지거나 커지는 현상
2. 훈련 데이터가 충분하지 않거나 레이블을 만드는 작업에 비용이 너무 많이 듦
3. 훈련이 극단적으로 느려질 수 있음
4. 수백만 개의 파라미터를 가진 모델은 훈련 세트에 과대적합될 위험이 큼. 훈련 샘플이 충분하지 않거나 잡음이 많은 경우 특히 더 그러함

## 1. 그레디언트 소실과 폭주 문제 해결

### 1-1) 적절한 초기화
- 입력과 출력의 분산을 동일하게, 역방향에서 층을 통과하기 전과 후의 그레이디언트 분산을 동일하게 하기 위해 가중치를 초기화
- 층의 입력과 출력 연결 개가 같지 않으면 이 두 가지를 보장할 수 없지만, 실전에서 매우 잘 작동한다고 입증된 대안들

#### fan-in, fan-out, fan-avg
- fan-in, fan-out : 층의 입력, 층의 출력의 개수
- fan-avg = (fan-in+fan-out)/2

### *잘못된 초기화 방법*
-  초기값을 모두 0으로 설정(0 이외에도 하나의 상수로 해도 마찬가지)
    - 각 층의 모든 뉴런이 같은 gradient를 가지기 때문에 같은 값을 나타냄. 이는 학습을 계속 진행해도 마찬가지
    - 이러한 학습은 각 뉴런으로 나누는 의미를 없앰
- 로지스틱 시그모이드 활성화 함수 + 가중치 초기화 방법(평균 : 0, 표준편차 : 1 -> 정규분포)을 사용하면 각 층에서 출력의 분산이 입력의 분산보다 더 큼
    - 점점 커지기 때문에 높은 층으로 갈 수록 활성화 함수의 출력이 0이나 1로 수렴
    - 따라서 gradient가 점점 0에 가까워짐.
    - 역전파가 되면서 0에 가까운 gradient가 점점 곱해지면서 gradient가 점점 작아지고, 아래쪽 층에는 아무 것도 도달하지 않게 됨
    - *sigmoid gradient : $y(1-y)$(항상 출력이 0~1사이)*

-> 가중치 초기화 표준편차를 1보다 작게 함으로써 layer을 거듭할 수록 오히려 출력의 표준편차가 작아지도록 조정!  
-> 그렇다면 표준편차를 어느정도로 설정해야 될 것인가

### Xavier initialization/Glorot initialization
- 평균이 0이고 분산이 $\sigma^{2} = 1/fan_{avg}$인 정규분포
- $r={(3\sigma^{2})}^{1/2}$일 때 -r과 +r 사이의 균등분포
- 활성화 함수 : 활성화 함수 X / 탄젠트 / 시그모이드
- sigmoid 함수와 같은 S자 함수의 경우 출력값이 표준정규분포를 띌 때 가장 안정적인 gradient를 가짐
    - relu의 경우 너무 표준편차가 작아지게 되면 출력값이 모두 0이 되면서 오히려 gradient가 0으로 사라지는 현상이 발생하기 때문에 , sigmoid에 비해 큰 표준편차가 필요

### He initialization
- 평균이 0이고 분산이 $\sigma^{2} = 2/fan_{in}$인 정규분포($fan_{out}/fan_{avg}$을 사용하는 경우도 있음)
- $r={(3\sigma^{2})}^{1/2}$일 때 -r과 +r 사이의 균등분포
- 활성화 함수 : ReLU 함수와 그 변종들

###  LuCun initialization
- 평균이 0이고 분산이 $\sigma^{2}= 1/fan_{in}$인 정규분포
- fan-in=fan-out이면 Xavier초기화와 같음
- 활성화 함수 : SELU

- 케라스는 기본적으로 균등분포의 글로럿(Xavier 초기화)를 사용(glorot_uniform)
    - keras.initializers.VarianceScaling(scale=1.,mode="fan_avg", distribution="uniform"
- he normal
    - keras.initializers.VarianceScaling(scale=1, mode="fan_in", distribution="truncated_normal")
    - truncated_normal = $\sigma^{2}=1.3xscale/mode$
    - untruncated_normal = $\sigma^{2}=scale/model$

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
# fan_in 기반의 He 초기화
keras.layers.Dense(10, activation="relu", kernel_initializer = "he_normal")

In [5]:
# fan_out 기반의 He 초기화
he_avg_init = keras.initializers.VarianceScaling(scale=2.,
                                mode="fan_avg",distribution="uniform")
keras.layers.Dense(10, activation="sigmoid", kernel_initializer = he_avg_init)

### 1-2) 적절한 활성화 함수

- **sigmoid** : 그레디언트 소실 문제
- **ReLU** : dying ReLU(훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않음)
    - 모든 샘플에 대한 입력의 가중치 합이 음수가 되면 뉴런이 죽음 -> 경사하강법이 더는 작동하지 않음
    - 첫 번째 은닉층이 아니라면 죽은 뉴런이 되살아날 수도 있음(경사하강법에 의해 이전 층의 뉴런을 바꾸어 죽은 뉴런의 입력에 대한 가중치 합이 양수가 될 수 있기 때문)
- **LeaklyReLU** : ReLU 함수의 변종, 거의 항상 relu보다 성능이 좋음
    - $LeaklyReLU_{a}(z)=max(az,z)$
    - a : z<0일 때의 기울기, 일반적으로 0.01 / 0.3 으로 설정(0.3이 더 성능이 좋은 경우가 많음)
- **RReLU(randomized leakly ReLU)** : 훈련하는 동안 주어진 범위에서 a를 무작위로 선택하고 테스트시에는 평균을 사용
    - 꽤 잘 작동, 과대적합의 위험을 줄여주는 규제의 역할을 하기도 함
- **PReLU(parametric leakly ReLU)** : a가 훈련하는 동안 학습됨.
    - 대규모 데이터셋에서는 성능이 높지만, 소규모 데이터셋의 경우 과적합의 우려가 있음
- **ELU(expotentional linear  unit)** : 훈련 시간이 줄고 성능도 높음
    - $ELU_{a}(z) = a(exp(z)-1), z<0$
    - $ELU_{a}(z) = z,           z>=0$
    - a : z가 큰 음숫값일 때 ELU가 수렴할 값을 정의(보통 1)
    - 장점
        - z<0이어도 그레디언트가 0이 아니므로 죽은 뉴런을 만들지 않음
        - a=1이면 z<0일 때의 도함수 : $a(exp(z))$이고, $z>=0$일 때 1 -> 모든 구간에서 연속적이어서 경사 하강법의 속도를 높여줌
        - z<0이면 음숫값이 출력되기 때문에 활성화 함수의 평균 출력이 0에 더 가까워짐. 이는 그레디언트 소실 문제를 완화시켜줌
    - 단점
        - 지수함수를 사용하기 때문에 계산이 느림. 훈련하는 동안에는 수렴 속도가 빨라서 느린 계산이 상쇄되지만 테스트 시에는 ReLU보다 느리게 됨
- **SELU(scaled ELU)** : 스케일이 조정된 ELU활성화 함수의 변종
    - 모든 은닉층이 SELU활성화 함수를 사용하면 네트워크가 자기 정규화됨
    - 훈련하는 동안 각 층의 출력이 평균 0과 표준편차 1을 유지하는 경향이 있음 -> 그레디언트 소실과 폭주 문제를 막아줌
    - 특히 깊은 네트워크에서 다른 활성화 함수보다 뛰어난 성능을 보임

*자기 정규화가 되기 위한 몇 가지 조건*
- 입력 특성이 반드시 표준화(평균 0, 표준편차 1)되어야 한다.
- 모든 은닉층의 가중치는 르쿤 정규분포(kernel_initializer = "lecun_normal") 초기화로 초기화되어야 한다.
- 네트워크는 일렬로 쌓은 층으로 구성되어야 한다. 스킵연결(ex 와이드 & 딥 네트워크)과 같은 순차적이지 않은 구조에서는 자기 정규화되는 것이 보장되지 않음. -> ELU가 더 성능이 나을 수도 있음.(SELU는 z=0에서 연속적이지 않기 때문)

#### 선택방법
    - 성능
        - SELU > ELU > LeaklyReLU > ReLU > tanh > logistic
        - 네트워크가 자기 정규화 되지 못하는 구조라면 SELU는 z=0에서 미분 불가능하기 때문에 ELU가 더 좋을 수도 있음
    - 속도
        1. ReLU : (지금까지) 가장 널리 사용되는 활성화 함수이므로 많은 라이브러리와 하드웨어 가속기가 ReLU에 특화되어 최적화 되어있음
        2. LeaklyReLU
    - 과적합
        1. RReLU
    - 대규모 데이터셋
        - PReLU

#### Leakly RELU

In [ ]:
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer = "he_normal"),
    keras.layers.LeaklyReLU(alpha = 0.2),
    [...]
])

#### PReLU

In [ ]:
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer = "he_normal"),
    keras.layers.PReLU(alpha = 0.2),
    [...]
])

#### SELU

In [ ]:
layer = keras.layers.Dense(10, activatino = "selu",
                          kernel_initializer = "lecun_normal")

### 1-3) 배치 정규화

- 각 층에서 활성화 함수를 통과하기 전이나 후에 연산을 추가
    - 전과 후 어디가 좋을 지는 논의 중
- 미니배치에서의 평균과 표준편차를 구함
- 대부분의 경우 신경망의 첫번째 층으로 배치 정규화를 추가하면 훈련 세트를 표준화할 필요가 없음
    - 배치 정규화 층이 이러한 역할을 대신 해주기 때문(배치마다 처리하기 때문에 근사적)
    - 입력 특성마다 스케일을 조정하고 이동할 수 있음
- 심층 신경망에서 매우 널리 사용하는 층

#### 방법
1. 평균을 구함
    - $\mu_{B} = \frac{1}{m_{B}} \sum_{i=1}^{m_{B}}(x^{(i)})$
1. 표준편차를 구함
    - $\sigma_{B}^{2} = \frac{1}{m_{B}} \sum_{i=1}^{m_{B}}(x^{(i)}-\mu_{B})^{2}$
1. 표준화
    - $\hat{x}^{(i)} = \frac{x^{i}-\mu_{B}}{(\sigma^{2}_{B}+\epsilon)^{1/2}}$
4. 스케일을 조정, 이동(두 개의 파라미터를 이용하여) -> 차원마다 각각 하나씩 가지고 있음
    - $z^{i} = \gamma\hat{x}^{(i)} + \beta$

#### 파라미터 최종 추정
- $\gamma$(출력 스케일 벡터), $\beta$(출력 이동 벡터) -> 일반적인 역전파를 통해 학습
- $\mu$(최종 입력 평균 벡터), $\sigma$(최종 입력 표준편차) -> non-trainable 파라미터(일반적인 역전파로 업데이트 되지 X)
    1. 훈련이 끝난 후 전체 훈련 세트를 신경망에 통과시켜 배치 정규화층의 각 입력에 대한 평균과 표준편차를 계산
    2. 배치로 훈련하는 동안 층의 입력 평균과 표준편차의 이동 평균을 사용해 훈련하는 동안 최종 통계를 추정
        - 이동 평균 시 v(momentum에서의 beta와 같음)의 경우 데이터셋이 크고 미니배치각 작으면 1에 더 가깝게 함
- 정규화할 축
    - 기본(axis=-1) : 차원별
        - 2차원 데이터(nxd)라면 d개의 평균, 표준편차, 스케일, 이동 파라미터
        - 3차원 데이터(nxhxw)f라면 w개의 평균, 표준편차, 스케일, 이동 파라미터
    - 각 샘플에 대해 하고 싶다면 axis=(1,2)(3차원이라면)

#### 효과
- gradient 소실 문제 해결
- 가중치 초기화에 네트워크가 훨씬 덜 민감
- 그레디언트 폭주 문제를 막아주기 때문에 learning rate를 크게 설정할 수 있음. -> 학습 속도를 높임
    - layer의 수가 증가하기 때문에 훈련은 더 느려질 수 있지만, 수렴이 훨씬 빨라지기 때문에 보통 상쇄됨. 따라서 더 적은 에포크로 동일한 성능에 도달 가능.
    - 대체로 훈련 시간이 보통 더 짧음
- 규제와 같은 역할을 하여 다른 규제 기법(ex dropout)의 필요성을 줄임
    - 전체 데이터셋이 아닌 미니배치마다 평균과 표준편차를 계산하기 때문에 훈련 데이터에 일종의 잡음을 넣는 것과 비슷. 이러한 잡음 덕분에 훈련 세트에 과대적합되는 것을 방지하는 규제의 효과를 가지며 미니배치의 크기가 클수록 효과는 줄어듦.
    - 규제는 부수효과이기 때문에 효과가 비교적 크지 않음. 규제를 위해서 드롭아웃과 같은 다른 규제 기법을 같이 활용하는 것이 좋음  

In [4]:
from tensorflow import keras

In [5]:
# 구현(활성화 함수 이후 batch normalization layer 추가)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation = "softmax")
])

In [6]:
model.summary()
# batch normalization layer의 파라미터 개수 : 차원 x 4

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [19]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [25]:
# 구현(활성화 함수 이전 batch normalization layer 추가)
# hidden layer에서 activation function을 지정하면 안됨
# batch noramlization은 이동 벡터를 포함하기 때문에 bias를 뺄 수 있음

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias = False, kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, use_bias = False, kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation = "softmax")
])

- 실제 keras의 BatchNormalization class의 경우 훈련과 평가 시에 다르게 작동
    - 훈련 시에는 배치 별로 BatchNormalization을 수행, 평가 시에는 최종 추정된 값으로 수행.
    - training=None 이라는 매개변수로 이를 조정(훈련 시에는 1)
    - keras.backend.learning_phase() : 훈련 시에는 1, 그 외에는 0을 반환하는 함수


### 1-4) 그레디언트 클리핑
- 그레디언트 클리핑(gradient clipping) : 그레디언트 폭주 문제를 완화하기 위해 역전파될 때 일정 임계값을 넘어서지 못하게 그레디언트를 잘라내는 방법
    - 순환 신경망(RNN)에 많이 적용됨
- 방법
    - 모든 편미분 값을 -a에서 a로 제한(a : 하이퍼파라미터)
        - gradient의 방향을 바꿀 수 있음
        - (0.9,100)의 벡터(두번째 축 방향) -> (0.9,1)(대각선 방향)
    - 벡터의 norm을 지정(norm : 하이퍼파라미터)
        - gradient의 방향을 바꾸지 않음
        - (0.9,100)의 norm을 1로 변경 -> (0.00899, 0.9999)
    - 두 방법 모두를 사용
        - 그레디언트가 크게 폭주하고 있을 경우 모두 사용하는 것도 좋음

In [26]:
# 그레디언트 벡터(learning rate를 곱하기 전)의 모든 원소를 -1~1 사이로 클리핑.
optimizer = keras.optimizers.SGD(clipvalue = 1.0) # 방향을 바꿀 수 있는 방법
optimizer = keras.optimizers.SGD(clipnorm = 1.0) # 방향을 바꾸지 않는 방법
optimizer = keras.optimizers.SGD(clipvalue = 1.0, clipnorm = 1.0) # 두 방법 모두 사용
model.compile(loss="mse", optimizer = optimizer)